In [1]:
import pandas as pd
import re
#set ups
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
# importing package 
import matplotlib.pyplot as plt 
import numpy as np 
import requests

#setups
%matplotlib inline

In [2]:
df=pd.read_csv("listings_full.csv")

## Doing all the checks needed

In [3]:
df.shape

(13182, 75)

In [4]:
df.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
source                                           object
name                                             object
description                                     float64
neighborhood_overview                            object
picture_url                                      object
host_id                                           int64
host_url                                         object
host_name                                        object
host_since                                       object
host_location                                    object
host_about                                       object
host_response_time                               object
host_response_rate                               object
host_acceptance_rate                            

In [5]:
# keeping the columns i am interested in
df=df[['id',
       'name', 
       'host_id',
       'host_name', 
       'host_since',
       'host_location', 
       'host_is_superhost',
       'host_neighbourhood', 
       'host_listings_count',
       'host_total_listings_count',
       'neighbourhood',
       'neighbourhood_cleansed',
       'latitude', 'longitude',
       'property_type',
       'room_type',
       'bathrooms_text',
       'beds',
       'price',
       'minimum_nights', 
       'maximum_nights',
       'minimum_minimum_nights',
       'maximum_minimum_nights',
       'minimum_maximum_nights',
       'maximum_maximum_nights',
       'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 
       'availability_365', 
       'review_scores_rating',
       'review_scores_accuracy',
       'license',
       'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms']].copy()

In [6]:
df.describe()

,id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,beds,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_365,review_scores_rating,review_scores_accuracy,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
count,1.318200e+04,1.318200e+04,13182.000000,13182.000000,13182.000000,13182.000000,13017.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,11075.000000,11075.000000,13182.000000,13182.000000,13182.000000,13182.000000
mean,3.954590e+17,2.166796e+08,20.644895,32.083447,37.980172,23.731728,2.071368,3.151267,571.466925,3.197390,4.595812,757.426567,781.464421,4.157988,767.141405,222.418146,4.737552,4.783382,12.034441,10.849416,1.031027,0.071082
std,4.270628e+17,1.732957e+08,61.423587,136.062916,0.013427,0.012480,1.412656,16.231489,457.941252,18.416376,26.587753,449.324853,437.633288,24.276247,440.906213,136.667965,0.431216,0.394101,19.431829,18.978576,4.439618,0.850407
min,1.059500e+04,1.805000e+03,1.000000,1.000000,37.950322,23.697700,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,3.166800e+07,6.022029e+07,1.000000,2.000000,37.969140,23.723970,1.000000,1.000000,90.000000,1.000000,1.000000,365.000000,365.000000,1.000000,365.000000,88.000000,4.680000,4.750000,1.000000,1.000000,0.000000,0.000000
50%,5.312285e+07,1.821563e+08,5.000000,6.000000,37.978573,23.729719,2.000000,2.000000,365.000000,2.000000,2.000000,1125.000000,1125.000000,2.000000,1125.000000,268.000000,4.860000,4.890000,4.000000,3.000000,0.000000,0.000000
75%,8.516889e+17,3.696750e+08,20.000000,23.000000,37.988298,23.737657,3.000000,2.000000,1125.000000,2.000000,3.000000,1125.000000,1125.000000,2.300000,1125.000000,348.000000,4.980000,5.000000,14.000000,12.000000,0.000000,0.000000
max,1.052831e+18,5.516695e+08,2444.000000,5529.000000,38.030830,23.780220,30.000000,800.000000,4444.000000,800.000000,1000.000000,4444.000000,4444.000000,800.000000,4444.000000,365.000000,5.000000,5.000000,121.000000,121.000000,44.000000,15.000000


In [7]:
# Ποιος είναι ο πιο συνηθισμένος τύπος δωματίου που συναντάμε στην πλατφρόμα;
df['room_type'].value_counts(normalize=True)*100

room_type
Entire home/apt    90.320134
Private room        8.276438
Hotel room          0.773782
Shared room         0.629646
Name: proportion, dtype: float64

### Πόσοι host έχουν πάνω από δύο καταχωρήσεις;

In [8]:
# κρατάμε τα ολόκληρα καταλύματα, καθώς αυτά συναντάμε συχνότερα
ent_house=df[df.room_type.str.contains('Entire',case=False)].copy().reset_index()

# κρατ΄αμε τα διαμερίσματα με ένα μπάνιο και δύο κρεβάτια
ent_house= ent_house.loc[(ent_house['bathrooms_text'].str.contains('1 ', na=False)) & (ent_house['beds'] == 2)]

# βρίσκω πόσα entire homes έχει ο κάθε host 
listing = pd.DataFrame(ent_house.host_id.value_counts(ascending=False)).copy().reset_index()

In [9]:
# κατηγοριοποιώ πόσα ακίνητα έχει ο κάθε host (οικοδεσπότης)
# με βάση την ελληνική νομοθεσία, από 3 και πάνω, η κατηγοριοποίηση αλλάζει
# αν έχουν πάνω από 3: 1
# αν έχουν κάτω από 3: 0
listing['many_properties']= listing['count'].apply(lambda x: 1 if x>2 else 0)

### Σε ποιές περιοχές έχουν περισσότερες καταχωρήσεις(listings);

In [10]:
# κρατάμε μόνο τις τιμές που είναι αληθείς 
# και τις χρησιμοποιούμε για τον εντοπισμό των host στο γενικό σύνολο δεδομένων
list_business_hosts = listing[listing['many_properties']>0].host_id.tolist()

In [11]:
# περνάς τη νέα μεταβλητή στο df σου
ent_house['many_properties'] = ent_house['host_id'].apply(lambda x: 1 if x in list_business_hosts else 0)

In [12]:
# Ποιο είναι το ποσοστό των κατοικιών που ανήκουν σε επαγγελματίες οικοδεσπότες ;
kat_prof=ent_house["many_properties"].value_counts(normalize=True)*100

### Καθαρίζουμε τις τιμές


In [13]:
from money_parser import price_str
from decimal import Decimal

In [14]:
def correct_prices(dirty_price):
    if not pd.isna(dirty_price): 
        amount_num = price_str(dirty_price) 
        amount_decimal = Decimal(str(amount_num)) 
        return float(amount_decimal) 

In [15]:
ent_house['cleaned_price'] = ent_house['price'].apply(lambda x: correct_prices(x))

### Mέση τιμή ανά διανυκτέρευση

In [16]:
# Για κάθε περιοχή ξεχωριστά

# ομαδοποίηση με βάση τις στήλες 'neighborhood_cleansed', 'many_properties' 
# και εφαρμόζουμε συγκεντρωτικά τις του μέσου όρου και της διάμεσου στις τιμές που καθαρίσαμε.
price_per_night_area=ent_house.groupby(by=['neighbourhood_cleansed', 'many_properties'])['cleaned_price'].agg(['median', 'mean']).reset_index()

In [17]:
# Για όλη την Αθήνα
price_per_night=ent_house.groupby(by=['many_properties'])['cleaned_price'].agg(['median', 'mean']).reset_index()

### Υπάρχει διαφορά τιμών μεταξύ  των διαμερισμάτων των εππαγελματιών και των ερασιτεχνών;

In [18]:
# κρατάμε το median των τιμών ανά περιοχή
price_per_night_area_med= price_per_night_area[['neighbourhood_cleansed','many_properties','median']].reset_index(drop=True).copy()

In [19]:
# ξαναστείνουμε το df ώστε να έχουμε σε μία στηλη τις τιμές όσων έχουν λιγότερα από 3 διαμερίσματα 
# και σε μία άλλη όσων εχουν περισσότερα
pay_comp=price_per_night_area_med.pivot_table(values='median', index='neighbourhood_cleansed', columns='many_properties').reset_index().copy()
pay_comp=pay_comp.rename(columns={0:'less_than_3', 
                                  1:'more_than_3'})

In [20]:
# βρίσκουμε τη διαφορά των τιμών ανά περιοχή 
# και στήνουμε το df ώστε να φαίνονται πρώτες οι περιοχές με τη μεγαλύτερη διαφορά
pay_comp['price_difference']= (pay_comp.more_than_3 - pay_comp.less_than_3 )
pay_comp = pay_comp.sort_values(by='price_difference', ascending=False).reset_index(drop=True).copy()

### Σε πόσες περιοχές τα διαμέρίσματα που ανήκουν σε επαγγελματίες οικοδεσπότες είναι πιο ακριβά;

In [21]:
#ξεχωρίζουμε τις διαφορές στις περιοχές που υπάρχει διαφορά 
# αν η διαφορά είναι πάνω από το 0, η τιμή των διαμερισμάτων των επαγγελματιών είναι υψηλότερη
# αν η διαφορά είναι πάνω από το 0, η τιμή των διαμερισμάτων των ερασιτεχνών είναι υψηλότερη 
#  αν δεν υπάρχει διαφορά δεν λαμβάνεται υπόψη
more=[]
less=[]

for i, item in pay_comp.iterrows():
    if item['price_difference'] > 0:
        more.append(item['neighbourhood_cleansed'])
    elif item['price_difference'] < 0:
        less.append(item['neighbourhood_cleansed'])

In [22]:
#  βρίσκουμε την διαφορά για κάθε περιοχή
pay_comp['percentage_difference'] = ((pay_comp['more_than_3'] - pay_comp['less_than_3']) / ((pay_comp['more_than_3'] + pay_comp['less_than_3']) / 2)) * 100
pay_comp=pay_comp.sort_values(by='percentage_difference', ascending=False)

# Ερωτήσεις

### 1) Πόσοι host έχουν πάνω από δύο καταχωρήσεις;

In [23]:
how_many_porperties= listing['many_properties'].value_counts(normalize=True)*100\

text= f' Στην πλατφόρμα του Airbnb, to {round(how_many_porperties[1])} των host(οικοδεσποτών) έχουν περισσότερα από 2 ακίνητα.'
text

' Στην πλατφόρμα του Airbnb, to 10 των host(οικοδεσποτών) έχουν περισσότερα από 2 ακίνητα.'

### 2) Πόσες καταχωρήσεις ανήκουν σε αυτούς τους host;

In [24]:
kat_prof=ent_house["many_properties"].value_counts(normalize=True)*100

In [25]:
owned_by_prof=kat_prof[1]
owned_by_amateurs=kat_prof[0]

text= f'Από τις κατοικίες που έχουν καταχωρηθεί στην πλατφόρμα, το {round(owned_by_prof)}% ανήκει σε επαγγελματίες οικοδεσπότες \
ένω το {round(owned_by_amateurs)}% ανήκει σε οικοδεσπότες χωρίς επαγγελματική δραστηριότητα.'

text

'Από τις κατοικίες που έχουν καταχωρηθεί στην πλατφόρμα, το 38% ανήκει σε επαγγελματίες οικοδεσπότες ένω το 62% ανήκει σε οικοδεσπότες χωρίς επαγγελματική δραστηριότητα.'

In [26]:
### Ποιο είναι το ποσοστό των κατοικιών που ανήκουν σε επαγγελματίες οικοδεσπότες ;

### 3) Ποιές είναι οι περιοχές με το μεγαλύτερο ποσοστό τέτοιου είδους καταχωρήσεων;

In [27]:
ent_house['neighbourhood_cleansed'].value_counts(normalize=True)*100

neighbourhood_cleansed
ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ              17.011360
ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ                  8.971745
ΝΕΟΣ ΚΟΣΜΟΣ                          8.767842
ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ             7.049228
ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ     6.233615
ΠΑΓΚΡΑΤΙ                             4.631518
ΑΜΠΕΛΟΚΗΠΟΙ                          4.107195
ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ                      3.932421
ΑΚΡΟΠΟΛΗ                             2.912904
ΘΗΣΕΙΟ                               2.738130
ΠΕΤΡΑΛΩΝΑ                            2.679872
ΣΤΑΔΙΟ                               2.330323
ΚΕΡΑΜΕΙΚΟΣ                           2.155549
ΙΛΙΣΙΑ                               1.893388
ΚΥΨΕΛΗ                               1.864259
ΓΟΥΔΙ                                1.835130
ΚΟΛΩΝΑΚΙ                             1.806001
ΝΙΡΒΑΝΑ                              1.689484
ΖΑΠΠΕΙΟ                              1.456452
ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ                     1.310807
ΠΑΤΗΣΙΑ                              1.310807
ΛΥΚΑΒΗΤΤΟΣ 

 ### 4) Ποια είναι η μέση τιμή ενός διαμερίσματος ανά ημέρα όταν νοικιάζεται από χρήστη που διαθέτει κάτω  από 3 ακίνητα (ερασιτέχνης) και τρία ή περισσότερα ακίνητα? (σε όλη την Αθήνα και ανά περιοχή). 

In [28]:
amateur_hosts = price_per_night['median'][0]
prof_hosts = price_per_night['median'][1]

precentage_difference=((prof_hosts-amateur_hosts)/((prof_hosts+amateur_hosts)/2))*100

text= f'Η μέση τιμή του διαμερίσματος ανά μέρα για τους οικοδεσπότες με 3 και πάνω ακίνητα είναι {prof_hosts} \
και για όσους έχουν έχουν κάτω απο 3 είναι {amateur_hosts}. Οι τιμές αυτές έχουν διαφορά {round(precentage_difference)}%'
text

'Η μέση τιμή του διαμερίσματος ανά μέρα για τους οικοδεσπότες με 3 και πάνω ακίνητα είναι 68.0 και για όσους έχουν έχουν κάτω απο 3 είναι 60.0. Οι τιμές αυτές έχουν διαφορά 12%'

In [29]:
text=f'Για κάθε περιοχή ξεχωριστά:'
text
pay_comp

many_properties,neighbourhood_cleansed,less_than_3,more_than_3,price_difference,percentage_difference
0,ΠΕΔΙΟ ΑΡΕΩΣ,50.0,99.0,49.0,65.771812
1,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,53.0,92.0,39.0,53.793103
2,ΝΕΑ ΚΥΨΕΛΗ,53.0,86.5,33.5,48.028674
3,ΖΑΠΠΕΙΟ,64.0,85.5,21.5,28.762542
5,ΛΥΚΑΒΗΤΤΟΣ,52.5,68.5,16.0,26.446281
6,ΣΤΑΘΜΟΣ ΛΑΡΙΣΗΣ,47.0,59.5,12.5,23.474178
8,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,46.0,58.0,12.0,23.076923
4,ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ,71.0,89.0,18.0,22.500000
9,ΚΟΛΩΝΟΣ,48.0,60.0,12.0,22.222222
15,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,46.0,57.0,11.0,21.359223


### 5) Ποιές είναι οι 5 περιοχές με τη μεγαλύτερη διαφορά μεταξύ αυτών των δύο τιμών. Σε πόσες περιοχές τα διαμερίσματα είναι πιο ακριβά; 

In [30]:
more_than_0= len(more)
less_than_0= len(less)

text= f'Σε {more_than_0} περιοχές της Αθήνας, τα διαμερίσματα που ανήκουν σε επαγγελματίες hosts είναι πιο ακριβά από \
ότι τα διαμερίσματα που ανήκουν σε ερασιτέχνες hosts που είναι ακριβότερα σε {less_than_0} περιοχές.'
text

'Σε 26 περιοχές της Αθήνας, τα διαμερίσματα που ανήκουν σε επαγγελματίες hosts είναι πιο ακριβά από ότι τα διαμερίσματα που ανήκουν σε ερασιτέχνες hosts που είναι ακριβότερα σε 12 περιοχές.'

In [31]:
areas_greatest_price_dif= ', '.join(pay_comp['neighbourhood_cleansed'].loc[:4].tolist())

text= f'Οι 5 γειτονιές με τη μεγαλύτερη διαφορά τιμών μεταξύ των επαγγελματιών host\
και των ερασιτεχνών είναι οι {areas_greatest_price_dif}'
text

'Οι 5 γειτονιές με τη μεγαλύτερη διαφορά τιμών μεταξύ των επαγγελματιών hostκαι των ερασιτεχνών είναι οι ΠΕΔΙΟ ΑΡΕΩΣ, 1Ο ΝΕΚΡΟΤΑΦΕΙΟ, ΝΕΑ ΚΥΨΕΛΗ, ΖΑΠΠΕΙΟ, ΛΥΚΑΒΗΤΤΟΣ, ΣΤΑΘΜΟΣ ΛΑΡΙΣΗΣ, ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ, ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ'